author: G.Seijo (giovanni.seijo@colorado.edu)

This code uses argovis (argovis.colorado.edu) APIs to query gridded Argo data (https://argovis.colorado.edu/docs/Argovis_User_Guide_Gridded_Products.html) \
from the Roemmich and Gilson Argo product (https://sio-argo.ucsd.edu/RG_Climatology.html). 

Because the queries are limited by size (determined by time and area) we split the query into yearly \
downloads and then concatenate into an xarray data set that is then saved in a netcdf for further use.

In [24]:
import requests, xarray, pandas, math, datetime
import numpy as np
from datetime import datetime, timedelta
import xarray as xr
from glob import glob
import os
import matplotlib.pyplot as plt

API_KEY='76d2fd7c7748d072c912163a5059d5f53e9e4475'

In [25]:
def xargrid(grid):
    # given the json response <grid> of a request to /grids,
    # return an xarray object with coordinates time, lat, lon, depth, and measurement value.
    
    lat = []
    lon = []
    time = []
    pres = []
    meas = []
    for p in grid[1:]:
        for i, e in enumerate(p['d']):
            lon.append(p['g']['coordinates'][0])
            lat.append(p['g']['coordinates'][1])
            # convert a string to a date (zeros to check at the end of date)
            time.append(datetime.strptime(p['t'], '%Y-%m-%dT%H:%M:%S.%fZ'))
            meas.append(p['d'][i])
            pres.append(grid[0]['levels'][i])
            
    df = pandas.DataFrame({"latitude": lat, 
                           "longitude": lon, 
                           "time": time, 
                           "pressure": pres, 
                           "measurement": meas}).set_index(["latitude","longitude","time","pressure"])
    return df.to_xarray()

In [32]:
#note: for a larger region, may need to partition the dates into smaller windows (i.e. monthly queries instead of yearly).
#define argo api params
grid_name = 'rgPsalTotal' #rgPsalTotal rgTempTotal
start_date = ['2004-01-01T00:00:00Z','2005-01-01T00:00:00Z','2006-01-01T00:00:00Z','2007-01-01T00:00:00Z','2008-01-01T00:00:00Z','2009-01-01T00:00:00Z','2010-01-01T00:00:00Z','2011-01-01T00:00:00Z','2012-01-01T00:00:00Z','2013-01-01T00:00:00Z','2014-01-01T00:00:00Z']
end_date = ['2004-12-31T00:00:00Z','2005-12-31T00:00:00Z','2006-12-31T00:00:00Z','2007-12-31T00:00:00Z','2008-12-31T00:00:00Z','2009-12-31T00:00:00Z','2010-12-31T00:00:00Z','2011-12-31T00:00:00Z','2012-12-31T00:00:00Z','2013-12-31T00:00:00Z','2014-12-31T00:00:00Z']
poly = '[[-80.00318,35.393312],[-80.169981,32.417542],[-72.5837,33.627231],[-64.824686,34.362607],[-56.975164,34.602016],[-49.127944,34.338179],[-41.375559,33.579111],[-33.799479,32.347111],[-34.049679,35.120904],[-41.521475,36.407097],[-49.196772,37.214404],[-56.985605,37.518256],[-64.784292,37.309037],[-72.487922,36.593391],[-80.00318,35.393312]]'
#north boudnary caribgrid '[[-80.00318,35.393312],[-80.169981,32.417542],[-72.5837,33.627231],[-64.824686,34.362607],[-56.975164,34.602016],[-49.127944,34.338179],[-41.375559,33.579111],[-33.799479,32.347111],[-34.049679,35.120904],[-41.521475,36.407097],[-49.196772,37.214404],[-56.985605,37.518256],[-64.784292,37.309037],[-72.487922,36.593391],[-80.00318,35.393312]]'
#east boundary caribgrid'[[-37.043984,-5.773026],[-33.743845,-5.698398],[-33.92796,34.423183],[-36.95536,34.075624],[-37.043984,-5.773026]]'
#central carib poly = '[[-70.022513,16.382647],[-69.997682,13.165754],[-62.548315,13.141574],[-62.622809,16.311165],[-70.022513,16.382647]]'
pres_range = '0,150'#'0,5'
outdir = '/glade/work/gseijo/obc_glorys/validation/'
ncname = 'argo_'+grid_name+'_north_boundary_carib025_2004_2014.nc'

In [33]:
for i in range(len(end_date)):
    params = {
      "gridName": grid_name,
      "startDate": start_date[i],
      "endDate": end_date[i],
      "polygon": poly,
      "presRange":pres_range
    }

    r = requests.get('https://argovis-api.colorado.edu/grids', params=params, headers={'x-argokey': API_KEY})
    rgdata = r.json()
    
    ds = xargrid(rgdata)
    
    if i == 0:
        ds_all = ds
    else:
        ds_all = xr.concat([ds_all,ds],dim='time')
ds.attrs['global_attribute'] = poly
ds_all.to_netcdf(outdir+ncname)